In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv2D, Flatten, Reshape, Conv2DTranspose
from keras.layers import LeakyReLU, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.datasets import cifar10
from google.colab import drive
drive.mount('/content/drive/')

IMPORTING THE CIFAR 10 DATASET

In [15]:
(x_train , y_train) , (x_test , y_test)  = cifar10.load_data()

REDUCING ALL PIXEL VALUES TO [0-1]


In [16]:
x_train  = x_train/255.00

THE DISCRIMINATOR FUNCTION

In [17]:
def get_discriminator(in_shape=(32,32,3)):
  model = Sequential()
  model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add(BatchNormalization()) 
  model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

CODE TO GET REAL IMAGE SAMPLES FROM A GIVEN DATASET

In [18]:
def generate_real_samples(dataset, n_samples):
  new_x = np.random.randint(0, dataset.shape[0], n_samples)
  X = dataset[new_x]
  y = np.ones((n_samples, 1))
  return X, y

CODE TO GENERATE FAKE SAMPLES

In [19]:
def generate_random_points(n_samples):
	x_input = np.random.randn(100 * n_samples)
	x_input = x_input.reshape(n_samples,100)
	return x_input

def generate_fake_samples(generator_model, n_samples):
	x_input = generate_random_points(n_samples)
	X = generator_model.predict(x_input)
	y = np.zeros((n_samples, 1))
	return X, y

CODE TO CREATE THE GENERATOR MODEL 

In [20]:
def get_generator():
  model = Sequential()
  n_nodes = 128 * 8 * 8
  model.add(Dense(n_nodes, input_shape = (100 ,)))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((8, 8, 128)))
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(3, (7,7), activation='sigmoid', padding='same'))
  return model

CODE TO CREATE A NEW MODEL CALLED GAN MODEL WHICH HOLDS THE GENERATOR AND DISCRIMINATOR MODELS IN SEQUENCE AND HELPS TO TRAIN AND IMPROVE THE GENRATOR

In [21]:
def define_gan(g_model, d_model):
  d_model.trainable = False
  model = Sequential()
  model.add(g_model)
  model.add(d_model)
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

CODE TO TRAIN THE GAN MODEL IN ORDER TO TRAIN AND IMPROVE THE GENERATOR

In [27]:
def train(g_model, d_model, gan_model, dataset, n_epochs=20, n_batch=256):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  for i in range(n_epochs):
    for j in range(bat_per_epo):
      X_real, y_real = generate_real_samples(dataset, half_batch)
      X_fake, y_fake = generate_fake_samples(g_model, half_batch)
      X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
      d_loss, _ = d_model.train_on_batch(X, y)
      X_gan = generate_random_points(n_batch)
      y_gan = np.ones((n_batch, 1))
      g_loss = gan_model.train_on_batch(X_gan, y_gan)
      print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))

    if (i+1) % 2 == 1:
      summarize_performance(i, g_model, d_model, dataset)

CODE TO SAVE AND SHOW A GENERATED IMAGE 

In [23]:
def save_plot(examples , epoch , n=10 ):
    for i in range(n*n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
	      
        plt.imshow(examples[i ,:, : ,:])
    filename = '/content/drive/My Drive/Colab Notebooks/plot_gan_optimized' +str(epoch)+ '.jpg'
	  
    plt.savefig(filename)
	  
    plt.close()   

CODE TO SUMMARIZE AND EVALUATE THE PERFORMANCE OF THE GAN MODEL

In [24]:
def summarize_performance(epoch, g_model, d_model, dataset, n_samples=100):
	X_real, y_real = generate_real_samples(dataset, n_samples)
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	x_fake, y_fake = generate_fake_samples(g_model, n_samples)
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100) , epoch)
	save_plot(x_fake, epoch)
	g_model.save('/content/drive/My Drive/Colab Notebooks/gan_optimized' +str(epoch)+ '.h5')

FINAL AND MAIN CODE TO EXECUTE ALL THE ABOVE FUNCTIONS IN THE EXACT ORDER

In [ ]:
d_model = get_discriminator()
g_model = get_generator()
gan_model = define_gan(g_model, d_model)
train(g_model, d_model, gan_model, x_train)

NOW WE LOAD THE TRAINED GENERATOR MODEL TO GENERATE IMAGES IN ORDER TO JUDGE ITS PERFORMANCE

In [ ]:
model = load_model('/content/drive/My Drive/Colab Notebooks/gan_optimized19.h5')

In [ ]:
model.compile(loss='binary_crossentropy')

In [ ]:
def save_plot(examples, n):
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i, :, :, 0])
	plt.show()

In [ ]:
def generate_random_points(n_samples):
	x_input = np.random.randn(100 * n_samples)
	x_input = x_input.reshape(n_samples, 100)
	return x_input

In [ ]:
random_points = generate_random_points(25)
x= model.predict(random_points)
save_plot(x, 5)       